Decentrathon 4.0_inDrive Case#2_Solved by Samat Kabdygali

Фаза 1: Пространственный Инжиниринг Признаков

In [1]:
!pip install haversine -q
import pandas as pd
import numpy as np
import os
from google.colab import drive
from haversine import haversine_vector, Unit
from tqdm.notebook import tqdm
# Настройки для лучшего отображения в ноутбуке
pd.set_option('display.float_format', lambda x: '%.4f' % x)
tqdm.pandas()

print("✅ Библиотеки успешно установлены и импортированы.")

✅ Библиотеки успешно установлены и импортированы.


In [2]:
# Монтируем Google Drive
try:
    drive.mount('/content/drive', force_remount=True)
    print("✅ Google Drive успешно подключен.")
except Exception as e:
    print(f"❗️ Ошибка при подключении Google Drive: {e}")

# --- ВАЖНО: Укажите ваш путь к файлу! ---
file_path = '/content/drive/MyDrive/Decetrathon 4.0_InDrive Case#2_Samat Kabdygali/geo_locations_astana_hackathon.csv' #

# Загружаем данные
if not os.path.exists(file_path):
    print(f"❗️ Файл не найден по пути: {file_path}")
else:
    print(f"Загружаем данные из файла: {file_path}...")
    df = pd.read_csv(file_path)
    print("✅ Данные успешно загружены.")
    print(f"Размер исходного датасета: {df.shape[0]} строк, {df.shape[1]} столбцов.")

Mounted at /content/drive
✅ Google Drive успешно подключен.
Загружаем данные из файла: /content/drive/MyDrive/Decetrathon 4.0_InDrive Case#2_Samat Kabdygali/geo_locations_astana_hackathon.csv...
✅ Данные успешно загружены.
Размер исходного датасета: 1262687 строк, 6 столбцов.


In [3]:
if 'df' in locals():
    print("Начинаем очистку данных...")
    initial_rows = df.shape[0]

    # Шаг 1: Удаляем строки с отрицательной скоростью (spd < 0)
    df_cleaned = df[df['spd'] >= 0].copy()

    # Шаг 2: Фильтруем строки с аномальной высотой (alt)
    rows_before_alt_clean = len(df_cleaned)
    df_cleaned = df_cleaned[(df_cleaned['alt'] >= 300) & (df_cleaned['alt'] <= 450)]

    print(f"Итоговый размер данных после очистки: {len(df_cleaned)} строк.")

    # --- КЛЮЧЕВОЕ ИСПРАВЛЕНИЕ: Сохраняем очищенный датасет со всеми точками ---
    output_dir = '/content/drive/MyDrive/Decetrathon 4.0_InDrive Case#2_Samat Kabdygali/processed_data/'
    os.makedirs(output_dir, exist_ok=True)
    cleaned_path = os.path.join(output_dir, 'geotracks_cleaned.parquet')

    try:
        df_cleaned.to_parquet(cleaned_path, index=False)
        print(f"✅ Очищенный датасет со всеми точками сохранен по пути:\n{cleaned_path}")
        print("Этот файл будет использоваться в Фазе 3 для гео-анализа.")
    except Exception as e:
        print(f"❗️ Не удалось сохранить очищенный файл. Ошибка: {e}")
else:
    print("❗️ DataFrame 'df' не найден.")



Начинаем очистку данных...
Итоговый размер данных после очистки: 1244607 строк.
✅ Очищенный датасет со всеми точками сохранен по пути:
/content/drive/MyDrive/Decetrathon 4.0_InDrive Case#2_Samat Kabdygali/processed_data/geotracks_cleaned.parquet
Этот файл будет использоваться в Фазе 3 для гео-анализа.


In [4]:
if 'df_cleaned' in locals():
    print("Начинаем вычисление и агрегацию пространственных признаков...")

    # --- Шаг 1: Пространственный инжиниринг ---
    df_features = df_cleaned.sort_values(by=['randomized_id']).reset_index(drop=True)
    df_features[['lat_next', 'lng_next']] = df_features.groupby('randomized_id')[['lat', 'lng']].shift(-1)
    df_features.dropna(subset=['lat_next'], inplace=True)

    distances = haversine_vector(
        list(zip(df_features['lat'], df_features['lng'])),
        list(zip(df_features['lat_next'], df_features['lng_next'])),
        unit=Unit.METERS
    )
    df_features['distance_m'] = distances + 1e-6

    delta_spd = df_features.groupby('randomized_id')['spd'].diff(-1).abs()
    df_features['speed_change_per_m'] = (delta_spd / df_features['distance_m']).fillna(0)

    delta_azm = df_features.groupby('randomized_id')['azm'].diff(-1).abs()
    delta_azm[delta_azm > 180] = 360 - delta_azm[delta_azm > 180]
    df_features['turn_rate_per_m'] = (delta_azm / df_features['distance_m']).fillna(0)

    delta_alt = df_features.groupby('randomized_id')['alt'].diff(-1).abs()
    df_features['alt_gradient'] = (delta_alt / df_features['distance_m']).fillna(0)

    # --- Шаг 2: Агрегация ---
    agg_dict = {
        'speed_change_per_m': ['mean', 'std', 'max', lambda x: x.quantile(0.95)],
        'turn_rate_per_m': ['mean', 'std', 'max', lambda x: x.quantile(0.95)],
        'alt_gradient': ['mean', 'max', lambda x: x.quantile(0.95)],
        'spd': ['mean', 'std']
    }
    df_aggregated = df_features.groupby('randomized_id').agg(agg_dict)
    df_aggregated.columns = [
        'd_speed_change_mean', 'd_speed_change_std', 'd_speed_change_max', 'd_speed_change_q95',
        'd_turn_rate_mean', 'd_turn_rate_std', 'd_turn_rate_max', 'd_turn_rate_q95',
        'd_alt_grad_mean', 'd_alt_grad_max', 'd_alt_grad_q95',
        'spd_mean', 'spd_std'
    ]

    def calculate_tortuosity(trip):
        total_distance = trip['distance_m'].sum()
        start_point = (trip['lat'].iloc[0], trip['lng'].iloc[0])
        end_point = (trip['lat_next'].iloc[-1], trip['lng_next'].iloc[-1])
        straight_distance = haversine_vector([start_point], [end_point], unit=Unit.METERS)[0] + 1e-6
        return total_distance / straight_distance

    tortuosity = df_features.groupby('randomized_id').progress_apply(calculate_tortuosity)
    df_aggregated['tortuosity'] = tortuosity

    df_aggregated.fillna(0, inplace=True)
    df_aggregated.replace([np.inf, -np.inf], 0, inplace=True)

    print("✅ Агрегация признаков завершена.")
    display(df_aggregated.head())
else:
    print("❗️ DataFrame 'df_cleaned' не найден.")


Начинаем вычисление и агрегацию пространственных признаков...


  0%|          | 0/6553 [00:00<?, ?it/s]

✅ Агрегация признаков завершена.


,d_speed_change_mean,d_speed_change_std,d_speed_change_max,d_speed_change_q95,d_turn_rate_mean,d_turn_rate_std,d_turn_rate_max,d_turn_rate_q95,d_alt_grad_mean,d_alt_grad_max,d_alt_grad_q95,spd_mean,spd_std,tortuosity
randomized_id,,,,,,,,,,,,,,
-9221304899272910788,0.0318,0.1048,1.1660,0.1138,0.5142,3.7661,53.6495,1.4086,0.0187,3.0848,0.0185,9.0847,6.1719,81.3129
-9217374206810770265,0.0346,0.0960,0.6709,0.1278,0.0053,0.0100,0.0587,0.0205,0.0077,0.1758,0.0316,11.1084,5.4371,20.2348
-9214548556609186054,0.0607,0.0868,0.3402,0.2624,0.2312,0.3136,1.4652,0.7700,0.0228,0.4672,0.0354,9.4865,5.5885,30.1768
-9214033164510198912,0.0440,0.1900,1.8164,0.0880,0.6474,4.5158,43.5268,0.9317,44086.0959,4100006.1035,0.0384,9.7529,5.8285,80.9293
-9212938812549517684,0.0223,0.0463,0.4535,0.0956,0.2604,0.9246,10.7662,0.9312,117482.5523,33600006.1035,0.0430,7.4027,5.4786,148.7896


In [5]:
if 'df_aggregated' in locals():
    # Создаем папку для сохранения, если ее нет
    output_dir = '/content/drive/MyDrive/Decetrathon 4.0_InDrive Case#2_Samat Kabdygali/processed_data/'
    os.makedirs(output_dir, exist_ok=True)

    output_path = os.path.join(output_dir, 'features_spatial.parquet')

    try:
        df_aggregated.to_parquet(output_path)
        print(f"✅ Финальная матрица признаков ('ДНК') успешно сохранена по пути:\n{output_path}")
        print("Этот файл будет использоваться в Фазе 2 для baseline-анализа.")
        print("\n--- Фаза 1 (Исправленная) успешно завершена! ---")
    except Exception as e:
        print(f"❗️ Не удалось сохранить файл. Ошибка: {e}")
else:
    print("❗️ DataFrame 'df_aggregated' для сохранения не найден.")


✅ Финальная матрица признаков ('ДНК') успешно сохранена по пути:
/content/drive/MyDrive/Decetrathon 4.0_InDrive Case#2_Samat Kabdygali/processed_data/features_spatial.parquet
Этот файл будет использоваться в Фазе 2 для baseline-анализа.

--- Фаза 1 (Исправленная) успешно завершена! ---


Фаза 2: Baseline-Анализ и Кластеризация

In [6]:
# Цель: Получить быстрый, надежный результат с помощью классических ML-методов.
# =====================================================================================
import pandas as pd
import numpy as np
import os
import joblib
from google.colab import drive

# ML-библиотеки из Scikit-learn
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.metrics import silhouette_score, calinski_harabasz_score

# Библиотеки для интерактивной визуализации
import plotly.express as px
import plotly.graph_objects as go

# --- Загрузка данных ---
try:
    drive.mount('/content/drive', force_remount=True)
except Exception as e:
    print(f"❗️ Ошибка при подключении Google Drive: {e}")

# Путь к файлу с признаками из Фазы 1
features_path = '/content/drive/MyDrive/Decetrathon 4.0_InDrive Case#2_Samat Kabdygali/processed_data/features_spatial.parquet'

if not os.path.exists(features_path):
    print(f"❗️ Файл не найден по пути: {features_path}")
    print("Пожалуйста, убедитесь, что вы выполнили код Фазы 1.")
else:
    print(f"Загружаем матрицу признаков из файла: {features_path}...")
    df_dna = pd.read_parquet(features_path)
    print("✅ Данные успешно загружены.")
    print(f"Размер датасета: {df_dna.shape[0]} поездок, {df_dna.shape[1]} признаков.")
    display(df_dna.head())

Mounted at /content/drive
Загружаем матрицу признаков из файла: /content/drive/MyDrive/Decetrathon 4.0_InDrive Case#2_Samat Kabdygali/processed_data/features_spatial.parquet...
✅ Данные успешно загружены.
Размер датасета: 6553 поездок, 14 признаков.


,d_speed_change_mean,d_speed_change_std,d_speed_change_max,d_speed_change_q95,d_turn_rate_mean,d_turn_rate_std,d_turn_rate_max,d_turn_rate_q95,d_alt_grad_mean,d_alt_grad_max,d_alt_grad_q95,spd_mean,spd_std,tortuosity
randomized_id,,,,,,,,,,,,,,
-9221304899272910788,0.0318,0.1048,1.1660,0.1138,0.5142,3.7661,53.6495,1.4086,0.0187,3.0848,0.0185,9.0847,6.1719,81.3129
-9217374206810770265,0.0346,0.0960,0.6709,0.1278,0.0053,0.0100,0.0587,0.0205,0.0077,0.1758,0.0316,11.1084,5.4371,20.2348
-9214548556609186054,0.0607,0.0868,0.3402,0.2624,0.2312,0.3136,1.4652,0.7700,0.0228,0.4672,0.0354,9.4865,5.5885,30.1768
-9214033164510198912,0.0440,0.1900,1.8164,0.0880,0.6474,4.5158,43.5268,0.9317,44086.0959,4100006.1035,0.0384,9.7529,5.8285,80.9293
-9212938812549517684,0.0223,0.0463,0.4535,0.0956,0.2604,0.9246,10.7662,0.9312,117482.5523,33600006.1035,0.0430,7.4027,5.4786,148.7896


In [7]:
if 'df_dna' in locals():
    print("Масштабируем признаки с помощью StandardScaler...")

    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(df_dna)

    # Сохраняем scaler для будущего использования (например, для обработки новых данных)
    output_dir = '/content/drive/MyDrive/Decetrathon 4.0_InDrive Case#2_Samat Kabdygali/processed_data/'
    scaler_path = os.path.join(output_dir, 'dna_scaler.gz')
    joblib.dump(scaler, scaler_path)

    print("✅ Признаки отмасштабированы.")
    print(f"✅ Scaler сохранен по пути: {scaler_path}")
else:
    print("❗️ DataFrame 'df_dna' не найден.")

Масштабируем признаки с помощью StandardScaler...
✅ Признаки отмасштабированы.
✅ Scaler сохранен по пути: /content/drive/MyDrive/Decetrathon 4.0_InDrive Case#2_Samat Kabdygali/processed_data/dna_scaler.gz


In [8]:
if 'features_scaled' in locals():
    print("Применяем PCA для снижения размерности...")

    # Мы оставим достаточно компонент, чтобы объяснить ~90% дисперсии
    pca = PCA(n_components=0.9, random_state=42)
    features_pca = pca.fit_transform(features_scaled)

    print(f"✅ PCA применен.")
    print(f"Исходное количество признаков: {features_scaled.shape[1]}")
    print(f"Количество главных компонент после PCA: {pca.n_components_}")
    print(f"Объясненная дисперсия: {pca.explained_variance_ratio_.sum():.2%}")

    # Визуализируем кумулятивную объясненную дисперсию
    fig = px.area(
        x=range(1, pca.n_components_ + 1),
        y=np.cumsum(pca.explained_variance_ratio_),
        labels={"x": "Количество компонент", "y": "Объясненная дисперсия"},
        title="Кумулятивная объясненная дисперсия по главным компонентам"
    )

    # --- ВОТ ИЗМЕНЕНИЕ ---
    # Добавляем темную тему, чтобы график стал цветным и на темном фоне
    fig.update_layout(template="plotly_dark+seaborn")

    fig.show()
else:
    print("❗️ Массив 'features_scaled' не найден.")

Применяем PCA для снижения размерности...
✅ PCA применен.
Исходное количество признаков: 14
Количество главных компонент после PCA: 7
Объясненная дисперсия: 90.41%


In [9]:
if 'features_pca' in locals():
    print("Кластеризуем поездки с помощью K-Means...")

    # Для хакатона выберем фиксированное число кластеров, например 4.
    # Этого достаточно для демонстрации разных стилей.
    N_CLUSTERS = 4

    kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(features_pca)

    # Добавляем метки кластеров в наш основной DataFrame
    df_dna['cluster'] = cluster_labels

    # Оцениваем качество кластеризации
    silhouette = silhouette_score(features_pca, cluster_labels)
    calinski = calinski_harabasz_score(features_pca, cluster_labels)

    print(f"✅ Кластеризация завершена. Обнаружено {N_CLUSTERS} кластеров.")
    print(f"  - Silhouette Score: {silhouette:.3f} (чем ближе к 1, тем лучше)")
    print(f"  - Calinski-Harabasz Score: {calinski:.0f} (чем выше, тем лучше)")

    # Визуализируем кластеры в 2D пространстве первых двух компонент PCA
    df_dna['pca_1'] = features_pca[:, 0]
    df_dna['pca_2'] = features_pca[:, 1]

    fig = px.scatter(
        df_dna, x='pca_1', y='pca_2', color='cluster',
        title='Визуализация кластеров стилей вождения (в пространстве PCA)',
        labels={'pca_1': 'Главная компонента 1', 'pca_2': 'Главная компонента 2'},
        category_orders={"cluster": [str(i) for i in range(N_CLUSTERS)]}
    )
    fig.show()

else:
    print("❗️ Массив 'features_pca' не найден.")


Кластеризуем поездки с помощью K-Means...
✅ Кластеризация завершена. Обнаружено 4 кластеров.
  - Silhouette Score: 0.726 (чем ближе к 1, тем лучше)
  - Calinski-Harabasz Score: 1901 (чем выше, тем лучше)


In [10]:
if 'features_scaled' in locals():
    print("Ищем аномальные поездки с помощью Isolation Forest...")

    # contamination - ожидаемая доля аномалий в данных. 1% - разумное значение.
    iso_forest = IsolationForest(contamination=0.01, random_state=42)
    anomaly_preds = iso_forest.fit_predict(features_scaled)

    # -1 означает аномалию, 1 - норму.
    df_dna['is_anomaly'] = anomaly_preds
    df_dna['is_anomaly'] = df_dna['is_anomaly'].map({1: 0, -1: 1}) # 1 - аномалия, 0 - норма

    num_anomalies = df_dna['is_anomaly'].sum()
    print(f"✅ Поиск аномалий завершен. Найдено {num_anomalies} аномальных поездок.")

    # Посмотрим на аномалии на PCA-графике
    df_dna['anomaly_label'] = df_dna['is_anomaly'].map({0: 'Норма', 1: 'Аномалия'})
    fig = px.scatter(
        df_dna, x='pca_1', y='pca_2', color='anomaly_label',
        title='Визуализация аномалий (в пространстве PCA)',
        labels={'pca_1': 'Главная компонента 1', 'pca_2': 'Главная компонента 2'},
        color_discrete_map={'Норма': 'blue', 'Аномалия': 'red'}
    )
    fig.show()

else:
    print("❗️ Массив 'features_scaled' не найден.")


Ищем аномальные поездки с помощью Isolation Forest...
✅ Поиск аномалий завершен. Найдено 66 аномальных поездок.


In [11]:
if 'cluster' in df_dna.columns:
    print("Интерпретируем результаты кластеризации...")

    # Убираем вспомогательные столбцы перед анализом
    df_to_analyze = df_dna.drop(columns=['pca_1', 'pca_2', 'anomaly_label'])

    # Группируем по кластерам и считаем средние значения признаков
    cluster_profiles = df_to_analyze.groupby('cluster').mean()

    print("\nПрофили кластеров (средние значения признаков):")
    display(cluster_profiles)

    # Для наглядности можно построить радарную диаграмму
    # Нормализуем данные для радара
    from sklearn.preprocessing import minmax_scale
    profiles_scaled = pd.DataFrame(minmax_scale(cluster_profiles.T).T,
                                     columns=cluster_profiles.columns,
                                     index=cluster_profiles.index)

    fig = go.Figure()
    for i in profiles_scaled.index:
        fig.add_trace(go.Scatterpolar(
            r=profiles_scaled.loc[i].values,
            theta=profiles_scaled.columns,
            fill='toself',
            name=f'Кластер {i}'
        ))

    fig.update_layout(
        title='Радарная диаграмма профилей стилей вождения',
        polar=dict(radialaxis=dict(visible=True, range=[0, 1])),
        showlegend=True
    )
    fig.show()

    # --- Сохранение ---
    output_dir = '/content/drive/MyDrive/Decetrathon 4.0_InDrive Case#2_Samat Kabdygali/processed_data/'
    output_path = os.path.join(output_dir, 'dna_results_baseline.parquet')

    try:
        df_to_analyze.to_parquet(output_path)
        print(f"✅ Финальные результаты (с кластерами и аномалиями) сохранены по пути:\n{output_path}")
        print("\n--- Фаза 2 успешно завершена! ---")
    except Exception as e:
        print(f"❗️ Не удалось сохранить файл. Ошибка: {e}")
else:
    print("❗️ DataFrame не содержит столбец 'cluster'.")


Интерпретируем результаты кластеризации...

Профили кластеров (средние значения признаков):


,d_speed_change_mean,d_speed_change_std,d_speed_change_max,d_speed_change_q95,d_turn_rate_mean,d_turn_rate_std,d_turn_rate_max,d_turn_rate_q95,d_alt_grad_mean,d_alt_grad_max,d_alt_grad_q95,spd_mean,spd_std,tortuosity,is_anomaly
cluster,,,,,,,,,,,,,,,
0,3.7381,34.1324,539.2314,6.4831,151217.3101,821883.4252,8553993.1762,327017.0305,6090.7831,541922.6446,12506.1043,7.5354,4.5088,766802565.5522,0.0043
1,20878.6055,97848.9422,907082.5000,111052.0552,6420851.6535,20560521.4638,118093385.0000,28458097.3750,164337.4724,1082740.0533,853226.7476,2.6114,1.6569,241.0019,1.0000
2,59.3329,403.3027,3356.5817,51.6093,9040822.0987,23488096.4767,131261766.7858,37532138.4247,152.1994,23313.5933,581.0443,4.3527,3.3814,9969235169.1731,0.0990
3,61.1553,128.6519,305.5760,305.1802,99.0672,299.7191,955.3591,528.5046,4506783.2844,39074998.4741,33651540.7577,2.5446,3.8385,9632579764.9881,1.0000


✅ Финальные результаты (с кластерами и аномалиями) сохранены по пути:
/content/drive/MyDrive/Decetrathon 4.0_InDrive Case#2_Samat Kabdygali/processed_data/dna_results_baseline.parquet

--- Фаза 2 успешно завершена! ---


In [12]:
!pip install h3pandas -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.2 MB/s eta 0:00:00


In [13]:
import pandas as pd
import numpy as np
import os
from google.colab import drive
import h3pandas
from haversine import haversine_vector, Unit
from sklearn.preprocessing import minmax_scale

# --- Загрузка данных ---
try:
    drive.mount('/content/drive', force_remount=True)
    print("✅ Google Drive успешно подключен.")
except Exception as e:
    print(f"❗️ Ошибка при подключении Google Drive: {e}")

# Путь к очищенному файлу из Фазы 1 (нужен для расчета спроса и рисков)
cleaned_data_path = '/content/drive/MyDrive/Decetrathon 4.0_InDrive Case#2_Samat Kabdygali/processed_data/geotracks_cleaned.parquet'

if not os.path.exists(cleaned_data_path):
    print(f"❗️ Файл с очищенными данными ({cleaned_data_path}) не найден.")
    print("Пожалуйста, убедитесь, что вы выполнили код Фазы 1.")
else:
    print(f"Загружаем очищенные данные из файла: {cleaned_data_path}...")
    df_points = pd.read_parquet(cleaned_data_path)
    print("✅ Очищенные данные успешно загружены.")


Mounted at /content/drive
✅ Google Drive успешно подключен.
Загружаем очищенные данные из файла: /content/drive/MyDrive/Decetrathon 4.0_InDrive Case#2_Samat Kabdygali/processed_data/geotracks_cleaned.parquet...
✅ Очищенные данные успешно загружены.


In [14]:
if 'df_points' in locals():
    print("Подготавливаем данные для гео-анализа...")
    # Этот код повторяет ключевые шаги из Фазы 1 для создания df_features

    df_features = df_points.sort_values(by=['randomized_id']).reset_index(drop=True)
    df_features[['lat_next', 'lng_next']] = df_features.groupby('randomized_id')[['lat', 'lng']].shift(-1)
    df_features.dropna(subset=['lat_next'], inplace=True)

    # Расчет расстояния
    distances = haversine_vector(
        list(zip(df_features['lat'], df_features['lng'])),
        list(zip(df_features['lat_next'], df_features['lng_next'])),
        unit=Unit.METERS
    )
    df_features['distance_m'] = distances + 1e-6

    # Расчет пространственных производных
    delta_spd = df_features.groupby('randomized_id')['spd'].diff(-1).abs()
    df_features['speed_change_per_m'] = (delta_spd / df_features['distance_m']).fillna(0)

    delta_azm = df_features.groupby('randomized_id')['azm'].diff(-1).abs()
    delta_azm[delta_azm > 180] = 360 - delta_azm[delta_azm > 180]
    df_features['turn_rate_per_m'] = (delta_azm / df_features['distance_m']).fillna(0)

    print("✅ Пространственные признаки для каждой точки готовы.")
else:
    print("❗️ DataFrame 'df_points' не найден.")


Подготавливаем данные для гео-анализа...
✅ Пространственные признаки для каждой точки готовы.


In [15]:
if 'df_features' in locals():
    print("Начинаем H3-индексацию и расчет Risk Score...")

    # 1. H3-индексация. Выбираем разрешение 9 (~175м в поперечнике)
    H3_RESOLUTION = 9
    df_h3 = df_features.h3.geo_to_h3(H3_RESOLUTION)

    print(f"✅ Данные проиндексированы с разрешением H3: {H3_RESOLUTION}")

    # 2. Агрегация поведенческих метрик по H3-ячейкам
    print("Агрегируем поведенческие метрики по ячейкам...")
    h3_agg = df_h3.groupby('h3_09').agg(
        # Берем 95-й перцентиль как показатель "экстремального" поведения
        p95_speed_change=('speed_change_per_m', lambda x: x.quantile(0.95)),
        p95_turn_rate=('turn_rate_per_m', lambda x: x.quantile(0.95)),
        # Доля остановок как показатель "дерганого" движения
        stop_density=('spd', lambda x: (x < 1).mean()),
        point_count=('randomized_id', 'count') # Количество точек в ячейке
    )

    # Отфильтровываем ячейки с малым количеством точек для надежности (k-анонимность)
    MIN_POINTS_PER_CELL = 10
    h3_risk = h3_agg[h3_agg['point_count'] >= MIN_POINTS_PER_CELL].copy()
    print(f"Оставлено {len(h3_risk)} ячеек после фильтрации по k-анонимности (min {MIN_POINTS_PER_CELL} точек).")

    # 3. Расчет формального Risk Score
    # Нормализуем каждый компонент риска от 0 до 1
    h3_risk['risk_speed'] = minmax_scale(h3_risk['p95_speed_change'])
    h3_risk['risk_turn'] = minmax_scale(h3_risk['p95_turn_rate'])
    h3_risk['risk_stop'] = minmax_scale(h3_risk['stop_density'])

    # Простое взвешивание компонентов (можно усложнить, но для хакатона это отлично)
    # Наибольший вес даем резким поворотам и изменениям скорости
    weights = {'speed': 0.4, 'turn': 0.4, 'stop': 0.2}
    h3_risk['risk_score'] = (
        weights['speed'] * h3_risk['risk_speed'] +
        weights['turn'] * h3_risk['risk_turn'] +
        weights['stop'] * h3_risk['risk_stop']
    )

    print("✅ Risk Score для каждой H3-ячейки успешно рассчитан.")
    display(h3_risk.sort_values('risk_score', ascending=False).head())

else:
    print("❗️ DataFrame 'df_features' не найден.")


Начинаем H3-индексацию и расчет Risk Score...
✅ Данные проиндексированы с разрешением H3: 9
Агрегируем поведенческие метрики по ячейкам...
Оставлено 70 ячеек после фильтрации по k-анонимности (min 10 точек).
✅ Risk Score для каждой H3-ячейки успешно рассчитан.


,p95_speed_change,p95_turn_rate,stop_density,point_count,risk_speed,risk_turn,risk_stop,risk_score
h3_09,,,,,,,,
89215383387ffff,0.0837,56038449.0000,0.3677,3079,0.1959,1.0000,0.6312,0.6046
8921538314bffff,0.3379,74.9980,0.5824,2596,1.0000,0.0000,1.0000,0.6000
89215380687ffff,0.2582,29.7714,0.3996,9060,0.7478,0.0000,0.6860,0.4363
8921538317bffff,0.1835,50.4780,0.5251,30556,0.5117,0.0000,0.9016,0.3850
892153833a3ffff,0.1745,61.5616,0.4956,20031,0.4830,0.0000,0.8509,0.3634


In [16]:
if 'df_points' in locals():
    print("Начинаем расчет Demand Score...")

    # 1. Находим последние точки каждой поездки
    trip_endpoints = df_points.loc[df_points.groupby('randomized_id')['spd'].idxmax()]

    # 2. Индексируем конечные точки по H3
    endpoints_h3 = trip_endpoints.h3.geo_to_h3(H3_RESOLUTION)

    # 3. Считаем количество поездок, закончившихся в каждой ячейке
    demand_counts = endpoints_h3.groupby('h3_09').size().reset_index(name='demand_raw')

    # 4. Нормализуем спрос от 0 до 1
    demand_counts['demand_score'] = minmax_scale(demand_counts['demand_raw'])

    # Устанавливаем H3-индекс в качестве индекса DataFrame для удобного объединения
    demand_counts.set_index('h3_09', inplace=True)

    print("✅ Demand Score для каждой H3-ячейки успешно рассчитан.")
    display(demand_counts.sort_values('demand_raw', ascending=False).head())

else:
    print("❗️ DataFrame 'df_points' не найден.")


Начинаем расчет Demand Score...
✅ Demand Score для каждой H3-ячейки успешно рассчитан.


,demand_raw,demand_score
h3_09,,
8921538310bffff,409,1.0000
89215380683ffff,338,0.8260
89215383317ffff,279,0.6814
89215383147ffff,279,0.6814
8921538068bffff,271,0.6618


In [17]:
if 'h3_risk' in locals() and 'demand_counts' in locals():
    print("Объединяем данные по риску и спросу...")

    # Объединяем два DataFrame по H3-индексу
    h3_final_grid = h3_risk.join(demand_counts, how='left').fillna(0)

    # Добавляем геометрию (полигоны) для каждой ячейки для отрисовки на карте
    h3_final_grid = h3_final_grid.h3.h3_to_geo_boundary()

    # Оставляем только нужные для дашборда столбцы
    columns_to_keep = [
        'risk_score', 'demand_score', 'demand_raw', 'point_count', 'geometry'
    ]
    h3_final_grid = h3_final_grid[columns_to_keep]

    # --- Сохранение ---
    output_dir = '/content/drive/MyDrive/Decetrathon 4.0_InDrive Case#2_Samat Kabdygali/processed_data/'
    output_path = os.path.join(output_dir, 'h3_grid_scores.geojson')

    try:
        # Сохраняем в формате GeoJSON, который идеально подходит для веб-карт
        h3_final_grid.to_file(output_path, driver='GeoJSON')
        print(f"✅ Финальный гео-слой с H3-ячейками сохранен по пути:\n{output_path}")
        print("\n--- Фаза 3 успешно завершена! ---")
        print("Мы готовы к созданию прототипа на Фазе 4.")
    except Exception as e:
        print(f"❗️ Не удалось сохранить файл. Ошибка: {e}")

    display(h3_final_grid.head())
else:
    print("❗️ Необходимые DataFrame для объединения не найдены.")


Объединяем данные по риску и спросу...
✅ Финальный гео-слой с H3-ячейками сохранен по пути:
/content/drive/MyDrive/Decetrathon 4.0_InDrive Case#2_Samat Kabdygali/processed_data/h3_grid_scores.geojson

--- Фаза 3 успешно завершена! ---
Мы готовы к созданию прототипа на Фазе 4.


,risk_score,demand_score,demand_raw,point_count,geometry
h3_09,,,,,
89215380603ffff,0.1059,0.2475,102.0000,8095,"POLYGON ((71.43299 51.09051, 71.43444 51.08876..."
8921538060bffff,0.1848,0.0637,27.0000,3381,"POLYGON ((71.43273 51.0871, 71.43417 51.08534,..."
89215380613ffff,0.1153,0.0147,7.0000,4558,"POLYGON ((71.4284 51.09236, 71.42984 51.09061,..."
89215380617ffff,0.1183,0.3701,152.0000,11862,"POLYGON ((71.43326 51.09393, 71.4347 51.09217,..."
8921538061bffff,0.3577,0.1569,65.0000,9561,"POLYGON ((71.42813 51.08895, 71.42957 51.08719..."
